In [2]:
# 필요한 라이브러리 설치
%pip install -q langchain langchain-openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


# CommaSeparatedListOutputParser

**CommaSeparatedListOutputParser**는 **쉼표로 구분된 목록을 반환할 때 유용한 출력 파서** 입니다.

이를 활용하면 **텍스트 데이터를 구조화된 리스트 형태로 쉽게 변환**할 수 있습니다.

In [3]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import csv
from pprint import pprint

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()


In [4]:
# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

# OpenAI 모델 설정
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0
)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser

# "AI 관련 기술"에 대한 체인 호출 실행
result = chain.invoke({"subject": "AI 관련 기술"})

# 쉼표로 구분된 리스트 출력
print(" AI 관련 기술 목록:")
print(result)

# 결과 활용 예시: CSV 파일로 저장
csv_filename = "./data/ai_technologies.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["AI 기술"])  # 헤더 추가
    for item in result:
        writer.writerow([item])

print(f" '{csv_filename}' 파일로 저장 완료!")


 AI 관련 기술 목록:
['Machine Learning', 'Deep Learning', 'Natural Language Processing', 'Computer Vision', 'Robotics']
 './data/ai_technologies.csv' 파일로 저장 완료!


In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import json

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과학 분야 전문가 AI입니다. 질문에 대해 체계적이고 간결한 답변을 JSON 형식으로 제공하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# JSON 출력 형식 지침을 프롬프트에 적용
prompt = prompt.partial(format_instructions=parser.get_format_instructions())



In [6]:
# OpenAI 모델 설정
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0
)

# 프롬프트, 모델, 출력 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 질문 설정 (우주 탐사 관련 질문)
question = "최근 10년간 진행된 주요 우주 탐사 미션 3가지를 알려주세요. \
각 미션의 이름은 `mission_name`에, 목표는 `goal`에, 주관 기관은 `agency`에 담아 주세요."

# 체인 실행 및 JSON 응답 받기
response = chain.invoke({"question": question})

# JSON 데이터 출력
print(json.dumps(response, indent=4, ensure_ascii=False))


[
    {
        "mission_name": "뉴호라이즌스",
        "goal": "명왕성 탐사",
        "agency": "NASA"
    },
    {
        "mission_name": "카시니-호이겐스",
        "goal": "토성의 위성 타이탄 탐사",
        "agency": "NASA, ESA, 이탈리아 우주국"
    },
    {
        "mission_name": "창어 4호",
        "goal": "달의 뒷면 탐사",
        "agency": "중국 우주국"
    }
]



### PandasDataFrameOutputParser

In [5]:
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import re

# Titanic 데이터셋 로드
df = pd.read_csv('data/titanic.csv')

# Pandas DataFrame Output Parser 설정
parser = PandasDataFrameOutputParser(dataframe=df)

# 형식 지침 출력
format_instructions = parser.get_format_instructions()
print("Format Instructions:\n", format_instructions)

Format Instructions:
 The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2,

In [6]:
# ChatOpenAI 모델 초기화
model = ChatOpenAI(temperature=0, model_name="gpt-4o")
# model = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
#     model="meta-llama/llama-4-scout-17b-16e-instruct",
#     temperature=0
# )

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template=""" 
    You are a helpful assistant that interacts with a Pandas DataFrame.
    The DataFrame contains the following columns: {columns}.
    
    Your task is to answer the user's query by generating a command in the following format:
    {format_instructions}
    
    User Query: {query}    
    """,
    input_variables=["query"],
    partial_variables={
        "format_instructions": format_instructions,
        "columns": ", ".join(df.columns)
    },
)
print(prompt.partial_variables['columns'])

Survived, Pclass, Name, Sex, Age, Siblings/Spouses Aboard, Parents/Children Aboard, Fare


In [10]:
# 체인 생성
chain = prompt | model | parser

# 모델 응답 받기
try:
    # **Name 열을 표시하십시오.**
    print('Name 컬럼 출력')
    df_query = "Show the Name column"

    parser_output = chain.invoke({"query": df_query})
    print(type(parser_output))
    print(parser_output)

        # **첫번째 행을 표시하십시오.**
    print('첫번째 행 출력')
    df_query2 = "첫번쟤 행(row)을 보여주세요."

    parser_output2 = chain.invoke({"query": df_query2})
    print(parser_output2)

except Exception as e:
    print(f"오류 발생: {e}")

Name 컬럼 출력
<class 'dict'>
{'Name': 0                                 Mr. Owen Harris Braund
1      Mrs. John Bradley (Florence Briggs Thayer) Cum...
2                                  Miss. Laina Heikkinen
3            Mrs. Jacques Heath (Lily May Peel) Futrelle
4                                Mr. William Henry Allen
                             ...                        
882                                 Rev. Juozas Montvila
883                          Miss. Margaret Edith Graham
884                       Miss. Catherine Helen Johnston
885                                 Mr. Karl Howell Behr
886                                   Mr. Patrick Dooley
Name: Name, Length: 887, dtype: object}
첫번째 행 출력
{'0': Survived                                        0
Pclass                                          3
Name                       Mr. Owen Harris Braund
Sex                                          male
Age                                          22.0
Siblings/Spouses Aboard          

In [13]:
import pandas as pd
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# OpenAI 모델 초기화
model = ChatOpenAI(temperature=0, model_name="gpt-4o")
# model = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
#     model="meta-llama/llama-4-scout-17b-16e-instruct",
#     temperature=0
# )

# 응답 스키마 정의  (data : {}, {},{} )
response_schemas = [
    ResponseSchema(name="data", description="A list of dictionaries representing table rows."),
]

# Output Parser 설정
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="""
    You are an AI assistant that generates tabular data. 
    You must return the data in JSON format that follows this schema:
    
    {format_instructions}
        
    **User Query:**
    {query}
    """,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
print(prompt)

input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"data": string  // A list of dictionaries representing table rows.\n}\n```'} template='\n    You are an AI assistant that generates tabular data. \n    You must return the data in JSON format that follows this schema:\n\n    {format_instructions}\n\n    **User Query:**\n    {query}\n    '


In [ ]:

# 체인 생성 (프롬프트 → 모델 → OutputParser)
chain = prompt | model | parser

# 실행 함수
def generate_dataframe(user_query):
    try:
        # 모델 호출
        json_response = chain.invoke({"query": user_query})

        # 모델이 반환한 JSON을 Pandas DataFrame으로 변환
        df = pd.DataFrame(json_response["data"])

        # 결과 출력
        print("\n🔹 Generated DataFrame:\n")
        return df

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None

In [21]:
# [예제 1] 2024년 상반기 서울 아파트 평균 매매 가격 데이터 생성
print('2024년 하반기 서울울 아파트 평균 매매 가격 데이터 생성')
df_seoul_housing = generate_dataframe(
    "Create a dataset of the average apartment sale prices in Seoul for the Second half of 2024 with columns: District (구), Average Price (in KRW), Number of Transactions, and Year-over-Year Change (%)."
)
print(df_seoul_housing.shape)
df_seoul_housing.head()

# gyeonggido
# seoul 서울
# first half <- 상반기 // second half <- 하반기기

2024년 하반기 서울울 아파트 평균 매매 가격 데이터 생성

🔹 Generated DataFrame:

(10, 4)


,District,Average Price,Number of Transactions,Year-over-Year Change (%)
0,Gangnam-gu,1500000000,1200,3.5
1,Jongno-gu,950000000,800,2.1
2,Mapo-gu,1100000000,950,4.0
3,Songpa-gu,1300000000,1100,3.8
4,Yongsan-gu,1400000000,700,2.5


In [24]:
print('2024년 서울 지하철역별 유동 인구 데이터')
# [예제 2] 2024년 서울 지하철역별 유동 인구 데이터
df_seoul_subway = generate_dataframe(
    "Generate a dataset of the top 10 busiest subway stations in Seoul in 2024 with columns: Station Name, Line Number, Daily Passenger Volume, and Weekday vs Weekend Ratio."
)
if df_seoul_subway is not None:
    # print(df_seoul_subway.shape)
    df_seoul_subway.head()

2024년 서울 지하철역별 유동 인구 데이터

🔹 Generated DataFrame:



In [25]:
print('한국 5대 편의점 브랜드별 2024년 매출 및 점포 수')
# [예제 3] 한국 5대 편의점 브랜드별 2024년 매출 및 점포 수
df_korean_convenience_stores = generate_dataframe(
    "Create a dataset of the top 5 convenience store brands in Korea in 2024 with columns: Brand Name, Number of Stores, Total Revenue (in billion KRW), and Market Share (%)."
)
df_korean_convenience_stores.head()

한국 5대 편의점 브랜드별 2024년 매출 및 점포 수

🔹 Generated DataFrame:



,Brand Name,Number of Stores,Total Revenue (in billion KRW),Market Share (%)
0,CU,15000,5000,35.0
1,GS25,14000,4800,33.5
2,7-Eleven,11000,3200,20.0
3,Emart24,5000,1500,7.5
4,Ministop,3000,800,4.0


### PydnticOutputPerser

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List

In [5]:
# 출력 구조를 정의하는 Pydantic 모델
class MovieRecommendation(BaseModel):
    movie_title: str = Field(description="추천 영화 제목")
    reason: str = Field(description="추천 이유")
    genre: List[str] = Field(description="영화 장르")
    estimated_rating: float = Field(description="10점 만점에서 예상 평점")
    
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=MovieRecommendation)

# 프롬프트 템플릿 설정
template = """
다음 사용자 요청에 따라 영화를 추천해주세요.
요청: {query}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)
print(prompt)

input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"movie_title": {"description": "추천 영화 제목", "title": "Movie Title", "type": "string"}, "reason": {"description": "추천 이유", "title": "Reason", "type": "string"}, "genre": {"description": "영화 장르", "items": {"type": "string"}, "title": "Genre", "type": "array"}, "estimated_rating": {"description": "10점 만점에서 예상 평점", "title": "Estimated Rating", "type": "number"}}, "required": ["movie_title", "reason", "genre", "estimated_rating"]}\n```'} messag

In [6]:
# ChatOpenAI 모델 초기화
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# 체인 구성 및 실행
query = "1990년대 클래식한 느낌의 공포 영화 추천해줘"
chain = prompt | model | parser
output = chain.invoke({"query": query})

# 결과 출력
print(f"추천 영화: {output.movie_title}")
print(f"추천 이유: {output.reason}")
print(f"장르: {', '.join(output.genre)}")
print(f"예상 평점: {output.estimated_rating}/10")

추천 영화: The Silence of the Lambs
추천 이유: 1990년대 클래식한 느낌의 공포 영화로, 긴장감이 넘치는 스릴러 영화입니다.
장르: 공포, 스릴러, 미스터리
예상 평점: 9.5/10


# StructuredOutputParser

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

# 출력 구조 정의 (평점, 장점, 단점, 요약)
response_schemas = [
    ResponseSchema(name="rating", description="5점 만점에서 예상 평점"),
    ResponseSchema(name="pros", description="리뷰에서 언급된 장점 3가지를 리스트로 출력"),
    ResponseSchema(name="cons", description="리뷰에서 언급된 단점 3가지를 리스트로 출력"),
    ResponseSchema(name="summary", description="리뷰를 한 문장으로 요약")
]

# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print("출력 형식 지시사항:")
print(format_instructions)

# 프롬프트 템플릿
template = """
다음 제품 리뷰를 분석하세요. 리뷰 내용: {review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

# 모델 초기화 (temperature=0.5로 설정해 일관성 있는 출력)
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)



출력 형식 지시사항:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"rating": string  // 5점 만점에서 예상 평점
	"pros": string  // 리뷰에서 언급된 장점 3가지를 리스트로 출력
	"cons": string  // 리뷰에서 언급된 단점 3가지를 리스트로 출력
	"summary": string  // 리뷰를 한 문장으로 요약
}
```


In [11]:
# 테스트 리뷰 데이터
review = """새로 구매한 스마트폰을 약 2주간 사용해본 결과, 전반적으로 성능과 완성도가 매우 인상적입니다. 우선 외관 디자인은 메탈 프레임과 유광 글래스 백 커버 조합으로 고급스러움을 잘 살렸고, 곡면 처리된 엣지 라인이 손에 착 감기는 그립감을 제공합니다. 무게는 약 180g 정도로 가볍고, 두께도 얇은 편이라 휴대성 면에서도 만족스럽습니다.
디스플레이는 6.7인치 AMOLED 패널을 탑재하고 있으며, 120Hz의 고주사율을 지원해 스크롤이나 애니메이션에서 매우 부드러운 경험을 제공합니다. FHD+ 해상도에 HDR10+를 지원해 색 표현력과 명암비가 뛰어나고, 실외 가시성을 높이기 위해 최대 밝기 약 1,500니트까지 지원되는 점도 매우 유용합니다.
카메라 시스템은 광각, 초광각, 망원 등 다중 카메라 구성을 갖추고 있고, 메인 센서는 5,000만 화소로 디테일 표현력이 뛰어납니다. 특히 야간 모드에서 이미지 프로세싱이 탁월하게 작동해, 저조도 환경에서도 노이즈가 적고 색감이 자연스럽습니다. OIS(광학 손떨림 보정) 기능이 적용되어 동영상 촬영 시 안정감이 크며, 4K 60fps 영상 촬영도 가능해 콘텐츠 제작용으로도 충분합니다. 전면 카메라도 1,200만 화소로, 피부톤 보정이 과하지 않아 자연스럽고 깔끔한 결과물을 보여줍니다.
퍼포먼스는 최신 Snapdragon 8 Gen 2 칩셋 기반으로, 앱 실행 속도나 멀티태스킹, 게임 플레이 시 전혀 끊김 없이 매우 쾌적합니다. RAM은 12GB로 넉넉하고, UFS 4.0 스토리지를 채택해 파일 전송 속도나 앱 로딩 속도도 빠릅니다. 장시간 고부하 작업 시에도 발열 제어가 잘 되어 있고, 써멀 스로틀링 현상도 거의 느껴지지 않았습니다.
배터리는 5,000mAh 용량으로 일반적인 사용 환경에서 하루 이상은 충분히 버팁니다. 특히 65W 고속 유선 충전과 30W 무선 충전을 모두 지원해 충전 효율이 우수하며, 30분 내외로 약 70% 이상 충전되는 것이 인상적입니다. 배터리 수명도 고려한 스마트 충전 알고리즘이 내장되어 장기적인 배터리 효율도 기대할 수 있습니다.
사운드 품질 역시 상위권입니다. 스테레오 듀얼 스피커를 탑재해 사운드 밸런스가 좋고, Dolby Atmos 지원으로 입체적인 음향 효과도 잘 구현됩니다. 통화 음질도 깨끗하고, 노이즈 캔슬링 기능이 포함된 마이크 덕분에 주변 소음이 많은 환경에서도 원활한 커뮤니케이션이 가능했습니다.
지문 인식 센서는 화면 내장형이며 반응 속도가 빠르고 정확도도 높은 편입니다. 얼굴 인식 기능도 병행 지원되며, 보안 수준도 나쁘지 않습니다. 운영체제는 최신 안드로이드 버전에 기반한 커스텀 UI로, 인터페이스가 직관적이고 군더더기 없이 정리돼 있어 사용자 경험이 깔끔합니다. 특히 불필요한 기본 앱이 거의 없고, 시스템 최적화가 잘 되어있다는 인상을 받았습니다.
방수·방진은 IP68 등급을 지원해 일상적인 물 튀김이나 먼지 걱정 없이 사용할 수 있고, 블루투스 5.3 기반의 연결도 안정적이며 빠릅니다. 다양한 서드파티 액세서리와의 호환성도 우수해 확장성 측면에서도 만족스럽습니다.
전반적으로 봤을 때, 이 스마트폰은 디자인, 디스플레이, 카메라, 성능, 배터리, 오디오 등 거의 모든 측면에서 균형 잡힌 완성도를 보여줍니다. 가격 대비 기능성과 성능이 매우 뛰어나며, 플래그십 모델다운 경험을 제공합니다. 
개인적으로는 현시점에서 가장 만족스러운 스마트폰 중 하나라고 생각합니다.

"""


# 이 스마트폰은 배터리 수명이 정말 좋아서 하루 종일 사용해도 충전이 필요 없었어요. 
# 카메라 화질도 선명하고, 특히 야간 모드가 훌륭합니다. 
# 다만 가격이 조금 비싸고, 무게가 200g이 넘어서 손이 피곤할 수 있어요.

# 체인 실행
chain = prompt | model | parser

output = chain.invoke({"review": review})

# 결과 출력 (Pretty Print)
print("===== 분석 결과 =====")
pprint(output)

===== 분석 결과 =====
{'cons': '',
 'pros': ['디자인, 디스플레이, 카메라, 성능, 배터리, 오디오 등 모든 측면에서 균형 잡힌 완성도를 보여줌',
          '가격 대비 기능성과 성능이 매우 뛰어남',
          '사용자 경험이 깔끔하고 직관적인 인터페이스 제공'],
 'rating': '5',
 'summary': '새로 구매한 스마트폰은 디자인, 성능, 카메라 등 모든 면에서 우수하며, 가격 대비 뛰어난 기능성과 성능을 제공하여 '
            '매우 만족스럽습니다.'}


In [14]:
#%pip install python-dateutil

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import DatetimeOutputParser
from datetime import datetime, timedelta

print(datetime.today())

# 출력 파서 초기화 (시간대 포함 가능)
datetime_parser = DatetimeOutputParser()
format_instructions = datetime_parser.get_format_instructions()

print("날짜 출력 형식 지시사항:")
print(format_instructions)

# 프롬프트 템플릿
# 현재 날짜를 명시적으로 프롬프트에 주입
current_date = datetime.now().strftime("%Y-%m-%d")
template = f"""
현재 날짜: {current_date}
다음 텍스트에서 날짜/시간 정보를 추출하세요. 상대적 표현(예: '다음 주 금요일')은 현재 날짜를 기준으로 계산합니다.
텍스트: {{text}}

{{format_instructions}}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

print(f'prompt = {prompt}')

2025-06-11 14:38:57.564765
날짜 출력 형식 지시사항:
Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 0026-05-18T01:29:55.659408Z, 1998-02-11T11:56:11.614090Z, 1701-12-24T23:49:21.587067Z

Return ONLY this string, no other words!
prompt = input_variables=['text'] input_types={} partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 0026-05-18T01:29:55.659408Z, 1998-02-11T11:56:11.614090Z, 1701-12-24T23:49:21.587067Z\n\nReturn ONLY this string, no other words!"} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], input_types={}, partial_variables={}, template="\n현재 날짜: 2025-06-11\n다음 텍스트에서 날짜/시간 정보를 추출하세요. 상대적 표현(예: '다음 주 금요일')은 현재 날짜를 기준으로 계산합니다.\n텍스트: {text}\n\n{format_instructions}\n"), additional_kwargs={})]


In [15]:
# 모델 초기화 (temperature=0.1로 설정해 정확한 날짜 출력 강조)
#model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.1
)

# 테스트 데이터 (다양한 날짜 형식 포함)
texts = [
    "회의는 2025년 6월 15일 오후 2시에 예정되어 있습니다.",
    "프로젝트 마감일은 다음 주 금요일입니다.",
    "행사 시작: 7/25/2025 18:00 KST",
    "3일 후에 시스템 점검이 진행됩니다."
]

# 체인 실행 및 결과 출력
chain = prompt | model | datetime_parser

for text in texts:
    print(f"\n원본 텍스트: {text}")
    output = chain.invoke({"text": text})
    print(f"추출된 날짜: {output.strftime('%Y-%m-%d %H:%M:%S %Z')}")

    



원본 텍스트: 회의는 2025년 6월 15일 오후 2시에 예정되어 있습니다.
추출된 날짜: 2025-06-15 14:00:00 

원본 텍스트: 프로젝트 마감일은 다음 주 금요일입니다.
추출된 날짜: 2025-06-20 00:00:00 

원본 텍스트: 행사 시작: 7/25/2025 18:00 KST
추출된 날짜: 2025-07-25 18:00:00 

원본 텍스트: 3일 후에 시스템 점검이 진행됩니다.
추출된 날짜: 2025-06-14 00:00:00 


In [21]:
# 이벤트 추출용 프롬프트
current_date = datetime.now().strftime("%Y-%m-%d")

event_template = """
현재 날짜: {current_date}
다음 텍스트에서 모든 이벤트의 날짜/시간을 추출하세요. 각 이벤트는 이름과 날짜를 포함해야 합니다.
각 이벤트의 날짜는 미래 일자를 기준으로 계산 합니다.
텍스트: {text}

출력 형식:
- 이벤트명: [이름]
- 날짜: [YYYY-MM-DD HH:MM:SS]
"""

event_prompt = ChatPromptTemplate.from_template(event_template)
event_chain = event_prompt | model
텍스트: {(text)}

# 예시 텍스트 (여러 이벤트 포함)
event_text = """
12월 10일에 크리스마스 마켓이 열리고, 12월 24일에는 크리스마스 이브 파티가 있습니다.
또한 1월 1일 00:00에 새해 카운트다운이 진행될 예정입니다.
"""

print(event_chain.invoke({"current_date":current_date, "text": event_text}).content)

현재 날짜: 2025-06-11

이벤트 목록:

- 이벤트명: 크리스마스 마켓
- 날짜: 2025-12-10 00:00:00

- 이벤트명: 크리스마스 이브 파티
- 날짜: 2025-12-24 00:00:00

- 이벤트명: 새해 카운트다운
- 날짜: 2026-01-01 00:00:00


# EnumOutputParser

In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

True

In [25]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum
from pprint import pprint

# 감정 클래스 정의 (Enum)
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "중립"

# EnumOutputParser 초기화
enumParser = EnumOutputParser(enum=Sentiment)
format_instructions = enumParser.get_format_instructions()

print("감정 분류 출력 형식:")
print(format_instructions)

# 프롬프트 템플릿
template = """
당신은 텍스트 감정 분석 전문가입니다.
다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

답변:"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)
print(prompt)

감정 분류 출력 형식:
Select one of the following options: 긍정, 부정, 중립
input_variables=['text'] input_types={} partial_variables={'format_instructions': 'Select one of the following options: 긍정, 부정, 중립'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], input_types={}, partial_variables={}, template='\n당신은 텍스트 감정 분석 전문가입니다.\n다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.\n\n텍스트: "{text}"\n\n{format_instructions}\n\n중요 규칙:\n1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요\n2. 다른 설명이나 부가 설명을 추가하지 마세요\n3. 이모지나 특수문자도 포함하지 마세요\n4. 오직 하나의 단어만 출력하세요\n\n답변:'), additional_kwargs={})]


In [40]:
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0  # 일관성을 위해 0으로 설정
)

# OutputFixingParser로 안정성 향상
fixing_parser = OutputFixingParser.from_llm(parser=enumParser, llm=model)

print("모델 및 파서 설정 완료")

# 테스트 텍스트
texts = [
    "이 제품 정말 좋아요! 완전 만족스러워요.",
    "서비스가 너무 느리고 불친절했습니다.",
    "오늘은 비가 온다네요.",
    "배송은 빠르지만 품질이 아쉽습니다.",
    "최고의 경험이었습니다!",
    "완전 실망했어요... 최악이에요",
    "일이 많아서 밤샘을 했어요.",
    "너무 맛 없어요.",
    "너무 맛 있었어요 다음에 또 올께요."
    
]

print(f"테스트할 텍스트 {len(texts)}개 준비 완료")

모델 및 파서 설정 완료
테스트할 텍스트 9개 준비 완료


In [41]:
# 안전한 감정 분석 함수 (에러 처리 포함)
def safe_sentiment_analysis(text, use_fixing_parser=True):
    """안전한 감정 분석 함수 - 에러 처리 포함"""
    try:
        # 기본 체인 생성
        chain = prompt | model | (fixing_parser if use_fixing_parser else enumParser)
        
        # 분석 실행
        result = chain.invoke({"text": text})
        return result, None
        
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."

# 실제 감정 분석 실행 (API 키 필요)
def run_sentiment_analysis():
    """실제 감정 분석 실행"""
    print("=== 실제 감정 분석 결과 ===")
    
    success_count = 0
    total_count = len(texts)
    
    for i, text in enumerate(texts, 1):
        print(f"\n{i}. 텍스트: {text}")
        
        # OutputFixingParser 사용
        result, error = safe_sentiment_analysis(text, use_fixing_parser=True)
        
        if result:
            print(f"   감정: {result.value} ")
            success_count += 1
        else:
            print(f"   오류: {error} ")
            
            # 기본 파서로 재시도
            print("   기본 파서로 재시도...")
            result2, error2 = safe_sentiment_analysis(text, use_fixing_parser=False)
            
            if result2:
                print(f"   감정: {result2.value} (기본 파서 성공)")
                success_count += 1
            else:
                print(f"   재시도 실패: {error2} ")
    
    print(f"\n=== 결과 요약 ===")
    print(f"성공: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
    print(f"실패: {total_count-success_count}/{total_count}")

# 실제 분석 실행 (API 키가 있는 경우)
try:
    run_sentiment_analysis()
except Exception as e:
    print("API 키가 설정되지 않았거나 네트워크 오류:")
    print("실제 실행을 위해서는 OpenAI API 키를 설정하세요.")
    print(f"오류 상세: {e}")

=== 실제 감정 분석 결과 ===

1. 텍스트: 이 제품 정말 좋아요! 완전 만족스러워요.
   감정: 긍정 

2. 텍스트: 서비스가 너무 느리고 불친절했습니다.
   감정: 부정 

3. 텍스트: 오늘은 비가 온다네요.
   감정: 중립 

4. 텍스트: 배송은 빠르지만 품질이 아쉽습니다.
   감정: 부정 

5. 텍스트: 최고의 경험이었습니다!
   감정: 긍정 

6. 텍스트: 완전 실망했어요... 최악이에요
   감정: 부정 

7. 텍스트: 일이 많아서 밤샘을 했어요.
   감정: 부정 

8. 텍스트: 너무 맛 없어요.
   감정: 부정 

9. 텍스트: 너무 맛 있었어요 다음에 또 올께요.
   감정: 긍정 

=== 결과 요약 ===
성공: 9/9 (100.0%)
실패: 0/9


In [46]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import BooleanOutputParser
from langchain_core.output_parsers import StrOutputParser

# Boolean 파서 초기화
boolParser = BooleanOutputParser()

# 수동으로 포맷 지시사항 정의 (LangChain 버전 이슈 회피)
format_instructions = """
출력은 반드시 다음 중 하나여야 합니다:
- `True`: 모든 조건 충족 시
- `False`: 하나라도 조건 불충족 시

예시:
True  # 모든 조건 만족
False # 조건 불만족
"""

# 승인/거부 결정 프롬프트 템플릿
template = """
다음 대출 신청자를 평가하세요. 조건을 모두 충족하면 `True`, 아니면 `False`를 출력하세요.

### 조건:
1. 나이 >= {min_age}세
2. 신용 점수 >= {min_credit_score}
3. 월 수입 >= ${min_income}

### 신청자 정보:
{applicant_details}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)
print(prompt)

input_variables=['applicant_details', 'min_age', 'min_credit_score', 'min_income'] input_types={} partial_variables={'format_instructions': '\n출력은 반드시 다음 중 하나여야 합니다:\n- `True`: 모든 조건 충족 시\n- `False`: 하나라도 조건 불충족 시\n\n예시:\nTrue  # 모든 조건 만족\nFalse # 조건 불만족\n'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['applicant_details', 'format_instructions', 'min_age', 'min_credit_score', 'min_income'], input_types={}, partial_variables={}, template='\n다음 대출 신청자를 평가하세요. 조건을 모두 충족하면 `True`, 아니면 `False`를 출력하세요.\n\n### 조건:\n1. 나이 >= {min_age}세\n2. 신용 점수 >= {min_credit_score}\n3. 월 수입 >= ${min_income}\n\n### 신청자 정보:\n{applicant_details}\n\n{format_instructions}\n'), additional_kwargs={})]


In [47]:
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.1
)

chain = prompt | model | boolParser

# 테스트 케이스
test_cases = [
    {
        "min_age": 18,
        "min_credit_score": 700,
        "min_income": 3000,
        "applicant_details": """
        - 이름: 김철수
        - 나이: 25세
        - 신용 점수: 750
        - 월 수입: $3,500
        """
    },
    {
        "min_age": 18,
        "min_credit_score": 700,
        "min_income": 3000,
        "applicant_details": """
        - 이름: 이영희
        - 나이: 17세
        - 신용 점수: 680
        - 월 수입: $2,800
        """
    }
]

# 거부 사유 생성 프롬프트 (출력 형식 명시적으로 지정)
reason_template = """
다음 대출 신청 거부 사유를 1문장으로 설명하세요. 반드시 다음 형식으로 답변해야 합니다:

[거부 사유]: [사유 내용]

### 신청자 정보:
{applicant_details}

### 조건:
- 최소 나이: {min_age}세
- 최소 신용 점수: {min_credit_score}
- 최소 월 수입: ${min_income}
"""
reason_prompt = ChatPromptTemplate.from_template(reason_template)
reason_chain = reason_prompt | model | StrOutputParser()

# 체인 확장 (Boolean 파서와 분리)
def get_decision_with_reason(input_dict):
    # 1. 먼저 Boolean 결정
    decision = chain.invoke(input_dict)
    
    # 2. 거부 시에만 사유 생성
    if not decision:
        try:
            reason = reason_chain.invoke({
                "applicant_details": input_dict["applicant_details"],
                "min_age": input_dict["min_age"],
                "min_credit_score": input_dict["min_credit_score"],
                "min_income": input_dict["min_income"]
            })
            return decision, reason
        except Exception as e:
            return decision, f"거부 사유 생성 실패: {str(e)}"
    return decision, "모든 조건을 충족했습니다."

# 테스트 (안전한 실행)
try:
    decision, reason = get_decision_with_reason(test_cases[0])
    print(f"\n결과: {'승인' if decision else '거부'}")
    print(f"사유: {reason}")
except Exception as e:
    print(f"오류 발생: {str(e)}")

오류 발생: BooleanOutputParser expected output value to include either YES or NO. Received ### 대출 신청자 평가

#### 조건:
1. 나이 >= 18세
2. 신용 점수 >= 700
3. 월 수입 >= $3000

#### 신청자 정보:
- 이름: 김철수
- 나이: 25세
- 신용 점수: 750
- 월 수입: $3,500

### 평가 코드

```python
def evaluate_loan_application(name, age, credit_score, monthly_income):
    # 조건 1: 나이 >= 18세
    condition1 = age >= 18
    
    # 조건 2: 신용 점수 >= 700
    condition2 = credit_score >= 700
    
    # 조건 3: 월 수입 >= $3000
    condition3 = monthly_income >= 3000
    
    # 모든 조건 충족 시 True 반환
    return condition1 and condition2 and condition3

# 신청자 정보
name = "김철수"
age = 25
credit_score = 750
monthly_income = 3500

# 평가 결과
result = evaluate_loan_application(name, age, credit_score, monthly_income)

print(result)  # True
```

### 결과 설명

- 나이: 25세 (조건 만족)
- 신용 점수: 750 (조건 만족)
- 월 수입: $3,500 (조건 만족)

모든 조건을 충족하므로 결과는 `True`입니다..
